In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
from scipy.signal import argrelextrema, find_peaks, peak_widths
from scipy.signal import savgol_filter
from pandas.plotting import autocorrelation_plot

In [ ]:
# Read in Data
df = pd.read_csv("path_to_your_data\\..\\..\\0Raising_the_power_front_Mocap.csv") #use appropriate path 

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
# Plot our data for a feature
x = df['timestamp']
name =  'lws_ydiff_xy' # 
y = df['marker_15_y'] - df['marker_11_y'] 
np.mean(y)


In [ ]:
def getInflectionPoint(x0, y0):
    # compute second derivative
    smooth_d2 = np.gradient(np.gradient(y0))
    # find switching points
    infls = np.where(np.diff(np.sign(smooth_d2)))[0]
    return x0[infls], y0[infls]

In [ ]:
# Inflection Point Selection
ys2 = savgol_filter(y, 4, 3)
ym2 = y.rolling(4).median()
yem2 = y.ewm(span=6).mean()
x0, y0 = getInflectionPoint(x, ys2)
plt.plot(x0, y0, '*k')
plt.plot(x, ys2)
plt.title('Data Smoothed with SG')
plt.figure()
x0, y0 = getInflectionPoint(x, y)
plt.plot(x0, y0, '*g')
plt.plot(x, y)
plt.title('Unsmoothed Data')
plt.figure()
x0, y0 = getInflectionPoint(x, ym2)
plt.plot(x0, y0, '*r')
plt.plot(x, ym2)
plt.title('MA')
plt.figure()
x0, y0 = getInflectionPoint(x, yem2)
plt.plot(x0, y0, '*r')
plt.plot(x, yem2)
plt.title('EMA')

In [ ]:
autocorrelation_plot(pd.Series(y))

In [ ]:
# Moving Average
y_ma_30 = y.rolling(30).mean() 
y_ema_30 = y.ewm(span=30).mean()
y_ma_15 = y.rolling(15).mean()
y_ema_10 = y.ewm(span=10).mean()
print(type(y_ema_30.array))
plt.plot(x,y)
plt.plot(x, y_ema_30)
#plt.plot(x, y_ma_30)
plt.plot(x,y_ema_10)
plt.legend(['reg', 'ema30', 'ema_10'])
plt.title(name)
plt.xlabel('Time (ms)')

In [ ]:
# Moving Average
y_ma_5 = y2.rolling(5).mean() 
y_ema_5 = y2.ewm(span=5).mean()
y_ma_1 = y2.rolling(2).mean()
y_ema_1 = y2.ewm(span=2).mean()
plt.plot(x2,y2)
plt.plot(x2, y_ma_5)
plt.plot(x2,y_ema_5)
plt.plot(x2, y_ma_1)
plt.plot(x2,y_ema_1)
plt.legend(['reg', 'ma5', 'ema5', 'ma2', 'ema2'])
plt.title(name2)
plt.xlabel('Time (ms)')

In [ ]:
# Savitsky Golay
# https://ieeexplore.ieee.org/abstract/document/5888646
ys0 = savgol_filter(y, 50, 0)
ys1 = savgol_filter(y, 50, 1)
ys2 = savgol_filter(y, 50, 2)
ys3 = savgol_filter(y, 50, 3)
ys4 = savgol_filter(y, 50, 4)
plt.plot(x,y)
plt.plot(x, ys0)
plt.plot(x, ys1)
plt.plot(x, ys2)
#plt.plot(x, ys3)
#plt.plot(x, ys4)
plt.legend(['reg', 'sg_0', 'sg_1', 'sg_2', 'sg_3', 'sg_4'])
plt.title(name)
plt.xlabel('Time (ms)')

In [ ]:
# Savitsky Golay
# https://ieeexplore.ieee.org/abstract/document/5888646
ys0 = savgol_filter(y2, 5, 0)
ys1 = savgol_filter(y2, 5, 1)
ys2 = savgol_filter(y2, 5, 2)
ys3 = savgol_filter(y2, 5, 3) #pref w/ window 5
ys4 = savgol_filter(y2, 5, 4)
plt.plot(x2,y2)
plt.plot(x2, ys0)
plt.plot(x2, ys1)
plt.plot(x2, ys2)
plt.plot(x2, ys3)
#plt.plot(x2, ys4)
plt.legend(['reg', 'sg_0', 'sg_1', 'sg_2', 'sg_3', 'sg_4'])
plt.title(name2)
plt.xlabel('Time (ms)')

In [ ]:
def selectIfGreater(x , y, val):
    nx = []
    ny = []
    for i in range(len(x)):
        if y[i] > val:
            nx.append(x[i])
            ny.append(y[i])
    return nx, ny

def selectIfLess(x , y, val):
    nx = []
    ny = []
    for i in range(len(x)):
        if y[i] < val:
            nx.append(x[i])
            ny.append(y[i])
    return nx, ny

In [ ]:
# refined extrema selection 
y_ema_30 = ys2
peaks, _ = find_peaks(y_ema_30)
true_peaks = argrelextrema(y_ema_30, np.greater)[0]
q75, q25 = np.percentile(y_ema_30, [75, 25])
mean_extrema = np.mean(y_ema_30)
peak_range = q75 - q25 # use iqr to filters
#(np.max(y_ema_30.values[peaks]) - np.min(y_ema_30.values[peaks])) / 1.5 # how should we determine denominator 
tpx, tpy = selectIfGreater(true_peaks, y_ema_30[true_peaks], mean_extrema)
true_trs = argrelextrema(y_ema_30, np.less)[0]
ttx, tty = selectIfLess(true_trs, y_ema_30[true_trs], mean_extrema)
results_half = peak_widths(y_ema_30, peaks, rel_height=0.75)
plt.figure()
plt.plot(y_ema_30)
plt.plot(peaks, y_ema_30[peaks], 'o')
plt.plot(true_peaks, y_ema_30[true_peaks], '*')
plt.plot(true_trs, y_ema_30[true_trs], '*')
plt.title('Feature' + ' \'' + name + '\'')
plt.xlabel('Time (ms)')
plt.legend(['Data', 'find_peaks', 'argr_greater', 'argr_less'])
plt.savefig('Data Points wo Filtering_' + name)
plt.figure()
plt.plot(y_ema_30)
plt.plot(tpx, tpy, "x")
plt.plot(ttx, tty, "*")
#plt.hlines(*results_half[1:], color="C2")
plt.legend(['Data', 'filt_argr_greater', 'filt_argr_less'])
plt.title('Feature' + ' \'' + name + '\'')
plt.xlabel('Time (ms)')
plt.savefig('Data Points w_Filtering_' + name)

In [ ]:
# FFT
X = np.fft.rfft(y)
ps = np.abs(X)**2
# Real fourier transform is taken to transform from the time to frequency domain
# Taking a real FFT removes the Herimitian symmetry, the negative frequency terms are just the complex
# conjugates of the corresponding positive-frequency terms, and the
# negative-frequency terms are therefore redundant.

N = len(X)
n = np.arange(N)
T = N/(1/0.06)
freq = n/T 

plt.figure(figsize = (12, 6))
plt.subplot(121)

plt.stem(freq, np.abs(X), 'b', \
         markerfmt=" ", basefmt="-b")
plt.xlabel('Freq (Hz)')
plt.ylabel('FFT Amplitude |X(freq)|')
print(freq[np.argmax(np.abs(X))])
ps = ps / np.sum(ps)
print(-1 / np.log2(N) * np.sum(ps * np.log2(ps)))

In [ ]:
# Sample Data
for i in [0, 5, 7]:
    if i != 24:
        path = "USE_THE_PATH_STORED" +  str(i) + "Raising_the_power_front_Mocap.csv" # root2
        df = pd.read_csv(path)
        for c in [15, 23, 25, 27]: #[0, 11, 12, 15, 16, 23, 24, 25, 26, 27, 28]:
            name = 'marker_' + str(c) + '_x'
            x = df['timestamp'][:50]
            y = df[name][:50]
            plt.figure()
            y_ma = y.rolling(4).mean()
            y_ema = y.ewm(span=6).mean()
            ys3 = savgol_filter(y, 6, 3)
            plt.plot(x, y)
            plt.plot(x, y_ma)
            plt.plot(x, y_ema)
            plt.plot(x, ys3)
            plt.title('Subject ' + str(i+1) + ' RTP')
            plt.ylabel(name)
            plt.xlabel('Time (ms)')
            plt.legend(['Raw', 'MA-4', 'EMA-6', 'SG-6-3'])
            plt.savefig('plots/../../' + str(i) + '_' + str(c) + '_' + name + '_RTP.png')
            

            name = 'marker_' + str(c) + '_y'
            x = df['timestamp']
            y = df[name]
            plt.figure()
            y_ma = y.rolling(10).mean()
            y_ema = y.ewm(span=10).mean()
            ys3 = savgol_filter(y, 20, 3)
            plt.plot(x, y)
            plt.plot(x, y_ma)
            plt.plot(x, y_ema)
            plt.plot(x, ys3)
            plt.title('Subject ' + str(i + 1) + ' RTP')
            plt.ylabel(name)
            plt.xlabel('Time (ms)')
            plt.legend(['Raw', 'MA-4', 'EMA-6', 'SG-6-3'])
            plt.savefig('plots/../../' + str(i) + '_' + str(c) + '_' + name + '_RTP.png')
        
        